In [1]:
import cv2
import mediapipe as mp

In [31]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

pose = mp_pose.Pose(static_image_mode=False)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks is not None:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(128, 0, 250), thickness=2, circle_radius=3),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

        frame = cv2.flip(frame, 1)
        height, width, _ = frame.shape

        landmarks = results.pose_landmarks.landmark
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
        right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE]
        left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE]

        right_wrist_coords = (int(right_wrist.x * width), int(right_wrist.y * height))
        left_wrist_coords = (int(left_wrist.x * width), int(left_wrist.y * height))
        right_ankle_coords = (int(right_ankle.x * width), int(right_ankle.y * height))
        left_ankle_coords = (int(left_ankle.x * width), int(left_ankle.y * height))

        cv2.putText(frame, f"M Derecha: {right_wrist_coords}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        cv2.putText(frame, f"M Izquierda: {left_wrist_coords}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.putText(frame, f"P Derecha: {right_ankle_coords}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.putText(frame, f"P Izquierda: {left_ankle_coords}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
